first version of read in file for the NMX work flow (from simulatet event data to binned event data)

this notebook takes a NEXUS file from a file_type simulation and performs the steps of binning and later one from Time of Arrival (TOA) to Time of Flight (TOF) and writes out a nexus file readable by DIALS (FormartNMX) 


first add neded libaries

In [1]:
from datetime import datetime
startTime = datetime.now()
import tracemalloc
    #from scippnexus import data
#import scippnexus as snx
import numpy as np
import math

    #qimport mantid_args
#from scippnexus import NXdetector
from scippneutron.conversion import graph

import scipp as sc
import scippneutron as scn
import scippnexus as snx

import xml.etree.ElementTree as ET

#%matplotlib widget


import plopp as pp
# pp.patch_scipp()
import matplotlib.pyplot as plt
from plopp.widgets import Box
import ipywidgets as ipw
from matplotlib.colors import LogNorm

    #import multiprocessing as mp

import h5py
h5py.enable_ipython_completer()
%matplotlib inline
%matplotlib widget

# starting the momory monitoring
tracemalloc.start()


define general parameter

In [3]:
#Number op pixels per detecotr dimmension
pix = 1280
pix_step = 100
#number of time bins 
t_step = 1

#number of detectors
n_det = 1
# normalising highest propability to 
max_prop = 100000

In [ ]:
#datapoints in output:
print("datapoint in output:",pix**2*3)

pretreatment to make it faster <br>
h5repack -l CHUNK=1024x6 2e11.h5 2e11-rechunk.h5  <br>
or <br>
h5repack -l CHUNK=NONE 2e11.h5 2e11-nochunk.h5<br>
or <br> 
h5repack -l CHUNK=1024x6 mccode.h5 mccode-nochunk.h5 

not neded with new McStas versions

test data filename must be the exact file to use for this step of data reduction

In [4]:
fname = 23

if fname == 1:
   filename = '/Users/justinbergmann/work_flow/test_data/2e11.h5'
elif fname == 2:
    filename = '/Users/justinbergmann/work_flow/test_data/x0_3d.h5'
elif fname == 3:
    filename = '/Users/justinbergmann/work_flow/test_data/xe_0_NMX.h5'
elif fname == 4:
    filename = '/Users/justinbergmann/work_flow/test_data/xe_20_NMX.h5'
elif fname == 5:
    filename = '/Users/justinbergmann/work_flow/test_data/flip_det.h5'
elif fname == 6:
    filename = '/Users/justinbergmann/work_flow/test_data/flip_4.h5'
elif fname == 7:
    filename = '/Users/justinbergmann/work_flow/test_data/ye_0_NMX.h5'
elif fname == 8:
    filename = '/Users/justinbergmann/work_flow/test_data/xe1_12_NMX/xe1_12_no_chunk.h5'
elif fname == 9:
    filename = '/Users/justinbergmann/work_flow/test_data/1d.h5'

elif fname == 10:
    filename = '/Users/justinbergmann/work_flow/test_data/xflat20.h5'

elif fname == 11:
    filename = '/Users/justinbergmann/work_flow/test_data/small_vis/flat.nxs'

elif fname == 12:
    filename = '/Users/justinbergmann/work_flow/test_data/small_vis/shift_down.nxs'


elif fname == 13:
    filename = '/Users/justinbergmann/work_flow/test_data/mc2_7_2_test.h5'

elif fname == 14:
    filename = '/Users/justinbergmann/work_flow/test_data/mc2_new_comp.nxs'

elif fname == 15:
    filename = '/Users/justinbergmann/work_flow/test_data/xflat20.h5'


elif fname == 16:
    filename = '/Users/justinbergmann/work_flow/test_data/flat_h_r.h5'


elif fname == 17:
    filename = '/Users/justinbergmann/work_flow/test_data/flat_v_u.h5'

elif fname == 18:
    filename = '/Users/justinbergmann/work_flow/test_data/NMX_long.h5'

elif fname == 19:
    filename = '/Users/justinbergmann/work_flow/test_data/pulse5.h5'



elif fname == 20:
    filename = '/Users/justinbergmann/work_flow/test_data/pulse5_y.h5'
elif fname == 21:
    filename = '/Users/justinbergmann/work_flow/test_data/wavelength.h5'

elif fname == 22:
    filename = '/Users/justinbergmann/work_flow/test_data/wavelength_short1.h5'
elif fname == 23:
    filename = '/Users/justinbergmann/work_flow/test_data/pulse5_z.h5'
elif fname == 24:
    filename = '/Users/justinbergmann/work_flow/test_data/nexus-v3_2-with_detectors_stageZ_and_one_axis5.h5'

else:
    filename = '/Users/justinbergmann/work_flow/test_data/xflat20.h5'


# f_vis = '/Users/justinbergmann/work_flow/test_data/ye_0_NMX_vis.nxs'
# f_vis ='/Users/justinbergmann/work_flow/test_data/mc2_new_comp_vis.nxs' 
# f_vis ='/Users/justinbergmann/work_flow/test_data/flat_v_u_vis.nxs'
f_vis ='/Users/justinbergmann/work_flow/test_data/pulse5_z_vis.nxs'  
print(filename)


#f=snx.File(filename)    

/Users/justinbergmann/work_flow/test_data/pulse5_z.h5


define needed fuctions

In [5]:
def CDist2(A,B):
    """
    Calculate the distance between two points.
    """
    dist = len3dvec(twoP_to_vec(A, B))
    return dist

def len3dvec(vec):
    """
    Calculate the length of a 3D vector.
    """
    a = np.sqrt(vec[0]**2 + vec[1]**2 + vec[2]**2)
    return a

def twoP_to_vec(A,B):
    """
    Create a vector between two points.
    """
    vec = np.array([B[0]-A[0], B[1]-A[1], B[2]-A[2]])
    return vec

def rotation_matrix(axis, theta):
    """
    Return the rotation matrix associated with counterclockwise rotation about
    the given axis by theta radians.
    """
    axis = np.asarray(axis)
    axis = axis / math.sqrt(np.dot(axis, axis))
    a = math.cos(theta / 2.0)
    b, c, d = -axis * math.sin(theta / 2.0)
    aa, bb, cc, dd = a * a, b * b, c * c, d * d
    bc, ad, ac, ab, bd, cd = b * c, a * d, a * c, a * b, b * d, c * d
    return np.array([[aa + bb - cc - dd, 2 * (bc + ad), 2 * (bd - ac)],
                     [2 * (bc - ad), aa + cc - bb - dd, 2 * (cd + ab)],
                     [2 * (bd + ac), 2 * (cd - ab), aa + dd - bb - cc]])

def get_data_path(file):
    """ try to find data in file"""
    file_type = "none"
    try:
        a = file['entry1/data']['bank01_events_dat_list_p_x_y_n_id_t']['events'][...]
        file_type = 'mcstas'
    except:
        try:
            a = file['entry1/data']['bank01_events_dat_list_p_x_y_n_id_t_L_L']['events'][...]
            file_type = 'mcstas_L'
        except:
            try:
                a = file['/entry/instrument/detector_panel_0/event_data/']['event_time_offset'][...]
                file_type = 'NMX_nexus'
            except:
                print("do data found in file")
                file_type = 'none'
    return a, file_type

def get_ids(file_type):
    """pixel IDs for each detector"""
    if file_type == 'mcstas' or file_type == 'mcstas_L':
        ids1 = sc.arange('id', 1, 1638401, unit=None)
        ids2 = sc.arange('id', 2000001, 3638401, unit=None)
        ids3 = sc.arange('id', 4000001, 5638401, unit=None)
        
    else:
        ids1 = sc.arange('id', 1, 1638401, unit=None)
        ids2 = sc.arange('id', 1638401, 3276802, unit=None)
        ids3 = sc.arange('id', 3276802, 4915203, unit=None)

    ids = sc.concat([ids1, ids2, ids3], 'id')
    return ids

def get_events_mcstas(a, file_type, max_prop):
    """get event list depending on dataformart and return dataset"""
    if file_type == 'mcstas' or file_type == 'mcstas_L':
        d = np.matrix.transpose(a)
        print("shape of event list (p_x_y_n_id_t)", d.shape)
        print("start extracting data")
        t_list = sc.array(dims=['x'], unit='s', values=d[5])
        id_list = sc.array(dims=['x'], unit=None, values=d[4], dtype='int64')
        weights = sc.array(dims=['x'], unit='counts', values=d[0]) #change to integer for measured data
        # Normalize weights to the maximum probability
        weights = weights * (max_prop/weights.max()) #delete for actual data
        # weights = sc.ones_like(x_list)
        # weights.unit = 'counts'
        print("tlist range",t_list.min(), t_list.max())
        da = sc.DataArray(data=weights, coords={ 't': t_list, 'id': id_list})
        # Ensure all IDs are recognized
        print("id min",id_list.values.min())
        print("id max",id_list.values.max())
    
    return da, weights



def read_in_data(filename):
    """
    Read data from an HDF5 file and prepare it for processing.
    """
    f = h5py.File(filename)
    a, file_type = get_data_path(file=f)
    if file_type == 'mcstas' or file_type == 'mcstas_L':
        # Allocate units to events and create separate lists for each parameter
        da, weights = get_events_mcstas(a, file_type, max_prop)
    elif file_type == 'NMX_nexus':
        da, weights = get_events_NMX_nexus(file=f)

    #print(x_list.shape, y_list.shape, t_list.shape,id_list.shape)

    

    ids = get_ids(file_type)
    
    return da, ids, file_type, weights

def get_instrument_geometry(filename,file_type):
    """ gether geometry data about the instrument"""
    if file_type== 'mcstas' or file_type=='mcstas_L':
        ds_l,fast_l,slow_l, sample_pos, source_pos = get_mcstas_geo(filename)

    else:
        ds_l,fast_l,slow_l, sample_pos, source_pos = get_default_parameters()

    print("sample position",sample_pos)
    print("source position",source_pos)
    print("relative to sample position",ds_l)
    print("fast axis",fast_l)
    print("slow axis", slow_l)
    return ds_l,fast_l,slow_l, sample_pos, source_pos


def get_events_NMX_nexus(file):
    """gether data for all detector panels """
    da_l =[]
    w_l = []
    for i in range(n_det):
        da , weights = get_events_NMX_nexus_panel(file,i)
        da_l .append(da)
        w_l.append(weights)
    data_l = sc.concat(da_l, 'x')
    waight_l = sc.concat(w_l, 'x')

    return data_l, waight_l
    
def get_events_NMX_nexus_panel(file, det_number):
    
    """get event list depending for NMX nexus formart"""
    d_path = '/entry/instrument/detector_panel_'+str(det_number)+'/event_data'
    print("data path",file[d_path]['event_id'])
    id_read = file[d_path]['event_id'][...]
    t_read = file[d_path]['event_time_offset'][...]
    print("event time offset",t_read)
    t0_index_read = file[d_path]['event_index'][...] 
    t0_read = file[d_path]['event_time_zero'][...]
    print("event_time_zero",t0_read)
    print("t",t_read.shape)
    print("t0",t0_read.shape)
    print("id",id_read.shape)
    t_d = t_read.astype(float)*10**9
    t0_d = t0_read.astype(float)
    id_d = id_read.astype(int)
    t0_index_d = t0_index_read.astype(int)
    print("t_d min max", min(t_d), max(t_d))
    print("t0_d min max", min(t0_d), max(t0_d))
    # t_d = np.matrix.transpose(t_read)
    #print("shape of event list (p_x_y_n_id_t)", d.shape)
    #print("start extracting data")
    # print("t_d",t_d.shape)

    ## transfere from event_time_offset, event_time_zero, event_index  to TOF

    t_list = sc.array(dims=['x'], unit='ns', values=t_d, dtype='int32')
    # t0_list = sc.array(dims=['x'], unit='ns', values=t0_d, dtype='int64')
    id_list = sc.array(dims=['x'], unit=None, values=id_d, dtype='int64')

    weights = sc.ones_like(t_list)
    weights.unit = 'counts'
    da = sc.DataArray(data=weights, coords={ 't': t_list, 'id': id_list})
    # Ensure all IDs are recognized
    #print("id min",id_list.values.min())
    #print("id max",id_list.values.max())
    

    return da, weights

In [ ]:
det_number = 1
d_path = '/entry/instrument/detector_panel_'+str(det_number)+'/event_data'
print(d_path)

In [6]:
def write_out_results(filename, ds_l,fast_l,slow_l, sample_pos, source_pos,group_t, t_step, weights ):

    #create name of outputfile
    no = filename.split('/')
    # print(no)
    name_out= no[-1].split('.')[0]
    print(name_out)
    file_out = '/Users/justinbergmann/work_flow/test_out/'+name_out+'_'+str(t_step)+'_out.h5'
    print(file_out)
    with h5py.File(file_out, 'w') as fo:
## create output nexus file
      fo.attrs['default'] = 'NMX_data'
      nxentry = fo.create_group('NMX_data')
      nxentry.attrs["NX_class"] = 'NXentry'
      nxentry.attrs['default'] = 'data'
      nxentry.attrs['name'] = 'NMX1'
      #nxentry.__setitem__('beamline','NMX')
      nxentry.__setitem__('name','NMX')
      nxentry.__setitem__('definition','TOFRAW')
      nxentry.attrs['name'] = "NMX"
      #nxentry.attrs['name'].__setattr__('name','NMX') 

#SAMPLE
      nx_sample = nxentry.create_group('NXsample')
      nx_sample.__setitem__('name','Single_crystal')


#Instrument
      nx_instrument = nxentry.create_group('NXinstrument')
 
      nx_detector = nxentry.create_group('NXdetector')
      det_origen = nx_detector.create_dataset('origen', data=ds_l) 
      det_origen.attrs['units'] = 'm'
 
      fast_axis = nx_detector.create_dataset('fast_axis', data=fast_l) 
      slow_axis = nx_detector.create_dataset('slow_axis', data=slow_l) 
 
      nx_beam = nxentry.create_group('NXbeam')
 
 
      
      proton = nxentry.create_dataset('proton_charge', data=weights.shape[0]/10000)    
      
      
      nx_det1 = nxentry.create_group('detector_1')   
      counts = nx_det1.create_dataset('counts', data=[group_t.values], compression="gzip", compression_opts=4)
 
      t_spectrum = nx_det1.create_dataset('t_bin', data=group_t.coords['t'].values, compression="gzip", compression_opts=4)
      t_spectrum.attrs['units'] = 'ms'
      t_spectrum.attrs['long_name'] = 't_bin TOF (ms)'
 
      pixel_id = nx_det1.create_dataset('pix_id', data=group_t.coords['id'].values, compression="gzip", compression_opts=4)
      pixel_id.attrs['units'] = ''
      pixel_id.attrs['long_name'] = 'pixel ID'


#SOURCE
      nx_inst = nxentry.create_group('instrument')
      nx_inst.attrs['nr_detector'] = len(fast_l)
      nx_source = nxentry.create_group('NXsource')
      nx_source.__setitem__('name','European Spallation Source')
      nx_source.__setitem__('short_name','ESS')
      nx_source.__setitem__('type','Spallation Neutron Source')
      nx_source.__setitem__('distance',-(CDist2(source_pos, sample_pos)))
      nx_source.__setitem__('probe','neutron')
      nx_source.__setitem__('target_material','W')






#    c_or = nxinst.create_dataset('crystal_orientation',data=cryst_or)
#    c_or.attrs['unit'] = 'degrees'
#    c_or.attrs['long_name'] = 'crystal orientation in Phi (degrees)'
    

    fo.close()
    del fo



    
    

In [7]:
def get_default_parameters():
    """ define standard parameter if no other parameters are specified"""

    source_pos = [0,0,0]
    sample_pos = [ -0.53123,   -0.,      -157.405  ]
    ds_l = [[ 0.25,  0.25, -0.29], [ 0.29,  0.25, -0.25], [-0.29,  0.25,  0.25]]
    fast_l = [[-1.  ,  0.  , -0.01], [-0.01,  0.  ,  1.  ], [ 0.01,  0.  , -1.  ]]
    slow_l = [[ 0., -1.,  0.], [ 0., -1.,  0.],[ 0., -1.,  0.]]
    return ds_l,fast_l,slow_l, sample_pos, source_pos


def get_mcstas_geo(filename):
    """ gets all gemoetry parameter from mcstas file"""

    det = False
    source = False
    sample = False
    sample_pos = [0,0,0]
    source_pos = [0,0,0]
    d_list = []
    rot_l1 = []
    fast_l = []
    slow_l = []

    f = h5py.File(filename)
    xml  = str(f['entry1/instrument/instrument_xml/data'][...][0]).split('\\n')
    comp = False



    for i in range(len(xml)):
        ls = xml[i].replace('<t',' ').replace('>',' ').replace('"',' ').replace('<',' ').replace('\\t',' ').split()
        # print(xml[i])
        
        if len(ls) >= 1:
            if ls[0] == 'component':
                det = False
                source = False
                sample = False
                comp = True
                if ls[2].split('-')[0] == 'MonNDtype':
 
                    det = True
                    d_list.append([int(ls[2].split('-')[1])])
                elif ls[2] == 'sourceMantid-type':
                    source = True
                elif ls[2] == 'sampleMantid-type':
                    sample = True
                comp = True
         #   if ls[1].split('-')[0] == 'type="MonNDtype':
                # print("1",ls)
        if len(ls) >= 1:
            if ls[0] == 'type':
                comp = False
        if len(ls) >= 1:
            if ls[0] == 'location' or ls[0] == 'location':
                # print("3",ls)
                if comp == True and det == True:
                    # print("2",ls)
                    xyz = [float(ls[2]),float(ls[4]),float(ls[6])]
                    # print("xyz", xyz)
                    rot =float(ls[8]) 
                    rot_xyz =[ float(ls[8]), float(ls[10]),float(ls[12]),float(ls[14])] 
                    # print("rotation of detector",rot, rot_xyz) 
                    d_list[len(d_list)-1].append(xyz)
                    rot_l1.append(rot_xyz)
                elif comp == True and source == True: 
                  source_pos = [float(ls[2]),float(ls[4]),float(ls[6])]
                elif comp == True and sample == True: 
                   sample_pos = np.array([float(ls[2]),float(ls[4]),float(ls[6])])
# print(len(d_list))
# print("sample and source position",sample_pos,source_pos)
# print("distance between sample and source",CDist2(source_pos, sample_pos))
# print("detector positions, relative to source at 0,0,0:",d_list)
# print("rotation list",rot_l)
#shift from rleative position to source to relative to sample
    ds_l = []
    sample_pos = sample_pos * [-1,-1,-1]
    # print("sample_pos",sample_pos)
    for i in range(len(d_list)):
        det_pos = np.array([d_list[i][1][0],d_list[i][1][1],d_list[i][1][2]]) * [-1.0, -1.0,-1.0]
        
 
        rel_xyz= np.round(twoP_to_vec(sample_pos,det_pos),2)
        # print("rel_xyz",rel_xyz)
        rel_xyz = rel_xyz # * [-1,-1,-1]
        # print("sample to detector dist",len3dvec(rel_xyz))
        # rel_pos= twoP_to_vec(det_pos, sample_pos)
        # print("detector position",sample_pos,det_pos,rel_pos)
        
        # rel_xyz = [ d_list[i][1][0]-sample_pos[0], d_list[i][1][1]-sample_pos[1], (d_list[i][1][2]-sample_pos[2])]
        
        # print("relative position",rel_xyz)
        # rel_xyz = [ sample_pos[0]-d_list[i][1][0],sample_pos[1]-d_list[i][1][1], sample_pos[2]-d_list[i][1][2]]
        # print("position",rel_xyz,rel_pos)
        # print("relative position",rel_xyz)
        ds_l.append(rel_xyz)
    vec_f = [-1,0,-0]
    vec_s = [0,-1,0]
    rot_l = [rot_l1[0],rot_l1[2],rot_l1[1]] #not corect but a temporary work around
    for i in range(len(rot_l)):
        # if rot_l[i][2] < 0:
            # theta = np.radians(rot_l[i][0]) 
        # else:
        theta = np.radians(-rot_l[i][0])
        v = [rot_l[i][1],rot_l[i][2],rot_l[i][3]]
        v1 = [0,0,1]
        # print("v",i,v,"theta", theta,rot_l[i][0])
        fast_vec = np.round(np.dot(rotation_matrix(v,  theta), vec_f),2)
        fast_v_round = np.array([np.round(fast_vec[0],1), np.round(fast_vec[1],1), np.round(fast_vec[2],1)])
 
        fast_l.append(fast_vec)
        slow_l.append(np.dot(rotation_matrix(v, theta), vec_s))
        # slow_l.append([0,1,0])

    # print("relative to sample position",ds_l)
    # print("fast axis",fast_l)
    # print("slow axis", slow_l)
    # print(rot_l)
    return ds_l,fast_l,slow_l, sample_pos, source_pos
    

In [ ]:
da, ids, file_type, weights  = read_in_data(filename)

3D visualisation with file converted by mantid (must be done before)

In [ ]:
grouped = da.group(ids)
del da , ids
grouped

In [ ]:
group_t = grouped.hist(t=t_step)
del grouped
group_t

In [8]:
%%time
#data reduction from event data to histogrammed data
#read in data from *.h5 file
da, ids, file_type, weights  = read_in_data(filename)
#grouping into pixel
grouped = da.group(ids)
del da, ids
#histogram in time per pixel
group_t = grouped.hist(t=t_step)
del grouped
# get instument geometry 
ds_l,fast_l,slow_l, sample_pos, source_pos = get_instrument_geometry(filename,file_type)
#write out to *h5 file wich is readable by DIALS with NMX class 
write_out_results(filename, ds_l,fast_l,slow_l, sample_pos, source_pos,group_t, t_step, weights) 

shape of event list (p_x_y_n_id_t) (6, 148001741)
start extracting data
tlist range <scipp.Variable> ()    float64              [s]  [0.0926507] <scipp.Variable> ()    float64              [s]  [0.1519]
id min 1
id max 5638399
sample position [  -0.53123   -0.      -157.405  ]
source position [0.0, 0.0, 0.0]
relative to sample position [array([ 0.25,  0.25, -0.29]), array([ 0.29,  0.25, -0.25]), array([-0.29,  0.25,  0.25])]
fast axis [array([-1.  ,  0.  , -0.01]), array([-0.01,  0.  ,  1.  ]), array([ 0.01,  0.  , -1.  ])]
slow axis [array([ 0., -1.,  0.]), array([ 0., -1.,  0.]), array([ 0., -1.,  0.])]
pulse5_z
/Users/justinbergmann/work_flow/test_out/pulse5_z_1_out.h5
CPU times: user 15.3 s, sys: 5.63 s, total: 21 s
Wall time: 8.22 s


Visualsisation after treatment with mantid 

Demo for STAP


In [9]:
%%time
f = h5py.File(filename)
a = f['entry1/data']['bank01_events_dat_list_p_x_y_n_id_t']['events'][...]
#a[0]
d = np.matrix.transpose(a)
print("shape of event list (p_x_y_n_id_t)", d.shape)
#alocate units to events and create seperate list for each parameter
x_list = sc.array(dims=['x'], unit='m', values=d[1])
y_list = sc.array(dims=['x'], unit='m', values=d[2])
t_list = sc.array(dims=['x'], unit='s', values=d[5])
id_list = sc.array(dims=['x'], unit=None, values=d[4], dtype='int64')
#print(x_list.shape, y_list.shape, t_list.shape,id_list.shape)


weights = sc.array(dims=['x'], unit='counts', values=d[0]) #change to integer for measured data
# normalise neutron with the highest probability to set value
weights = weights * (max_prop/weights.max()) #delete for actual data
# weights = sc.ones_like(x_list)
# weights.unit = 'counts'
da = sc.DataArray(data=weights, coords={'x': x_list, 'y': y_list, 't': t_list, 'id': id_list})

#make sure alll IDs are reconised:
print("id min",id_list.values.min())
print("id max",id_list.values.max())

ids1 = sc.arange('id', 1, 1638401, unit=None)
ids2 = sc.arange('id', 2000001, 3638401, unit=None)
ids3 = sc.arange('id', 4000001, 5638401, unit=None)
ids = sc.concat([ids1, ids2, ids3], 'id')
#grouping by IDs
da3 = da.group(ids).fold('id', sizes={'panel':3, 'id':-1})

shape of event list (p_x_y_n_id_t) (6, 148001741)
id min 1
id max 5638399
CPU times: user 17 s, sys: 8.99 s, total: 26 s
Wall time: 11.4 s


In [10]:
#binning of each detector panel in t_step bins
panel_0 = da3['panel',0].hist(t=t_step)
panel_1 = da3['panel',1].hist(t=t_step)
panel_2 = da3['panel',2].hist(t=t_step)

p0 = pp.slicer(da3.fold('id', sizes={'ypix':pix, 'xpix':-1})['panel', 0].hist(t=t_step).transpose(['t','ypix','xpix']),vmax=panel_0.max().value, aspect='equal',title='panel 0')
p1 = pp.slicer(da3.fold('id', sizes={'ypix':pix, 'xpix':-1})['panel', 1].hist(t=t_step).transpose(['t','ypix','xpix']),vmax=panel_1.max().value, aspect='equal',title='panel 1')
p2 = pp.slicer(da3.fold('id', sizes={'ypix':pix, 'xpix':-1})['panel', 2].hist(t=t_step).transpose(['t','ypix','xpix']),vmax=panel_2.max().value, aspect='equal',title='panel 2')

#p0.children[0].ax.set_ylim(1280, 0)
#p1.children[0].ax.set_ylim(1280, 0)
#p2.children[0].ax.set_ylim(1280, 0)

p0.children[0].ax.set_xlim(1280, 0)
p1.children[0].ax.set_xlim(1280, 0)

p2.children[0].ax.set_xlim(1280, 0)

l1 = ipw.jslink((p0.children[1].children[0].children[1], 'value'),
               (p1.children[1].children[0].children[1], 'value'))
l2 = ipw.jslink((p0.children[1].children[0].children[1], 'value'),
               (p2.children[1].children[0].children[1], 'value'))
Box([[p2, p0, p1]])

Box(children=(HBar(children=(Box(children=(InteractiveFig(children=(HBar(), HBox(children=(VBar(children=(Tool…

In [11]:
panel0_events = da3['panel', 0].bins.concat().values
nbin= pix
t_step = 250
# nbin = 250
binned = panel0_events.bin(y=nbin, x=nbin)
#binned = da.bin(y=50, x=50)
#binned = da.bin(y=225, x=225)
#sc.transpose(binned)

binned

hist3=binned.hist(t=t_step)

hist4 = hist3.copy()
for name in list(hist4.coords.keys()):
    hist4.coords[f'{name}c'] = sc.midpoints(hist4.coords[name])
hist4

hist5 = sc.flatten(hist4, to='row')
ava = np.average(hist5.values) #avarage of values to use as cut off creteria
sig = np.std(hist5.values)
cut = ava + sig*15
#print("background substraction",cut, ava, sig)
select = hist5.data > sc.scalar(cut, unit='dimensionless') # cut of background
filtered = hist5[select]
filtered.coords['tc'] *= 10
filtered

pp.scatter3d(filtered, x='xc', y='yc', z='tc', pixel_size=0.001,cmap='Greys_r',opacity=0.5)#,figsize=(550, 550))

Box(children=(Figure(children=(HBar(children=(HTML(value=''),)), HBox(children=(VBar(children=(Toolbar(childre…

In [13]:
# Select ranges
da = hist5
da.coords['tc'] *= 10
# cut0 = ava + sig*20
cut1 = ava + sig*40
cut2 = ava + sig*15
a = da[da.data > sc.scalar(cut1)]
b = da[(da.data > sc.scalar(cut2)) & (da.data < sc.scalar(cut1))]
# print(a,b)

# Display both on the same scatter plot
p = pp.scatter3d({'a': a, 'b': b},  x='xc', y='yc', z='tc', norm='log', pixel_size=0.001)

# Extract the point clouds from the final plot and set a lower opacity on the second point cloud
clouds = list(p[0].artists.values())
clouds[1].opacity = 0.05

p

Box(children=(Figure(children=(HBar(children=(HTML(value=''),)), HBox(children=(VBar(children=(Toolbar(childre…

In [ ]:
vis = sc.io.open_hdf5(filename = f_vis)
scn.instrument_view(vis.sum('tof'))

read in event data and shape the event list to give the number of events and creates the dataset for further modification

In [ ]:
%%time
f = h5py.File(filename)
a = f['entry1/data']['bank01_events_dat_list_p_x_y_n_id_t']['events'][...]
#a[0]
d = np.matrix.transpose(a)
print("shape of event list (p_x_y_n_id_t)", d.shape)
#alocate units to events and create seperate list for each parameter
x_list = sc.array(dims=['x'], unit='m', values=d[1])
y_list = sc.array(dims=['x'], unit='m', values=d[2])
t_list = sc.array(dims=['x'], unit='s', values=d[5])
id_list = sc.array(dims=['x'], unit=None, values=d[4], dtype='int64')
#print(x_list.shape, y_list.shape, t_list.shape,id_list.shape)


weights = sc.array(dims=['x'], unit='counts', values=d[0]) #change to integer for measured data
# normalise neutron with the highest probability to set value
weights = weights * (max_prop/weights.max()) #delete for actual data
# weights = sc.ones_like(x_list)
# weights.unit = 'counts'
da = sc.DataArray(data=weights, coords={'x': x_list, 'y': y_list, 't': t_list, 'id': id_list})

#make sure alll IDs are reconised:
print("id min",id_list.values.min())
print("id max",id_list.values.max())

ids1 = sc.arange('id', 1, 1638401, unit=None)
ids2 = sc.arange('id', 2000001, 3638401, unit=None)
ids3 = sc.arange('id', 4000001, 5638401, unit=None)
ids = sc.concat([ids1, ids2, ids3], 'id')
#grouping by IDs
da3 = da.group(ids).fold('id', sizes={'panel':3, 'id':-1})


does the same as above but with a reading file that contains also wavelength record at the detector

In [ ]:
%%time
f = h5py.File(filename)
a = f['entry1/data']['bank01_events_dat_list_p_x_y_n_id_t_L_L']['events'][...]
#a[0]
d = np.matrix.transpose(a)
print("shape of event list (p_x_y_n_id_t)", d.shape)
#alocate units to events and create seperate list for each parameter
x_list = sc.array(dims=['x'], unit='m', values=d[1])
y_list = sc.array(dims=['x'], unit='m', values=d[2])
t_list = sc.array(dims=['x'], unit='s', values=d[5])
id_list = sc.array(dims=['x'], unit=None, values=d[4], dtype='int64')
lam_list=sc.array(dims=['x'], unit='A', values=d[6])
#print(x_list.shape, y_list.shape, t_list.shape,id_list.shape)

#get evetns to seperated lists (x,y,t,id)
#x_list.unit = 'm'
#y_list.unit = 'm'
#t_list.unit = 'ms'
weights = sc.array(dims=['x'], unit='counts', values=d[0])
weights = weights * (100000/weights.max())
# weights = sc.ones_like(x_list)
# weights.unit = 'counts'
da = sc.DataArray(data=weights, coords={'x': x_list, 'y': y_list, 't': t_list, 'id': id_list, 'labda': lam_list})

#make sure alll IDs are reconised:
print("id min",id_list.values.min())
print("id max",id_list.values.max())

ids1 = sc.arange('id', 1, 1638401, unit=None)
ids2 = sc.arange('id', 2000001, 3638401, unit=None)
ids3 = sc.arange('id', 4000001, 5638401, unit=None)
ids = sc.concat([ids1, ids2, ids3], 'id')
#grouping by IDs
da3 = da.group(ids).fold('id', sizes={'panel':3, 'id':-1})



min and max time in the read in file

In [ ]:
print("t min and t max",t_list.min(), t_list.max())

In [ ]:
print("lambda min and labda max in A",lam_list.min(), lam_list.max())

plotting the three detectors with changeable TOF component

In [ ]:
#binning of each detector panel in t_step bins
panel_0 = da3['panel',0].hist(t=t_step)
panel_1 = da3['panel',1].hist(t=t_step)
panel_2 = da3['panel',2].hist(t=t_step)

p0 = pp.slicer(da3.fold('id', sizes={'ypix':pix, 'xpix':-1})['panel', 0].hist(t=t_step).transpose(['t','ypix','xpix']),vmax=panel_0.max().value, aspect='equal',title='panel 0')
p1 = pp.slicer(da3.fold('id', sizes={'ypix':pix, 'xpix':-1})['panel', 1].hist(t=t_step).transpose(['t','ypix','xpix']),vmax=panel_1.max().value, aspect='equal',title='panel 1')
p2 = pp.slicer(da3.fold('id', sizes={'ypix':pix, 'xpix':-1})['panel', 2].hist(t=t_step).transpose(['t','ypix','xpix']),vmax=panel_2.max().value, aspect='equal',title='panel 2')

#p0.children[0].ax.set_ylim(1280, 0)
#p1.children[0].ax.set_ylim(1280, 0)
#p2.children[0].ax.set_ylim(1280, 0)

p0.children[0].ax.set_xlim(1280, 0)
p1.children[0].ax.set_xlim(1280, 0)

p2.children[0].ax.set_xlim(1280, 0)

l1 = ipw.jslink((p0.children[1].children[0].children[1], 'value'),
               (p1.children[1].children[0].children[1], 'value'))
l2 = ipw.jslink((p0.children[1].children[0].children[1], 'value'),
               (p2.children[1].children[0].children[1], 'value'))
Box([[p2, p0, p1]])

grouping data of each pixel to prepare for time binning (these are the data that get used to write out)

In [ ]:
grouped = da.group(ids)
del da , ids
grouped


binning data in a predefined number of time binns

In [ ]:
group_t = grouped.bins.constituents['data'].hist(t=t_step)
del grouped
group_t

In [ ]:
group_t = grouped.hist(t=t_step)
group_t

needed functions to define instrument geometry

In [ ]:
t_edges = sc.linspace('t', t_list.min().value, t_list.max().value, t_step+1, unit=t_list.unit)
pix_min = ids.min().value
pix_max = ids.max().value
print("pix_min, pix_max",pix_min, pix_max)
# pix_bin = grouped[0:100]
# pix_bin.hist(t=t_edges)
# pix_grop = grouped[0:1]
# pix_hist = pix_grop.hist(t=t_edges)
step_data = pix_hist
pix_per_step=int(pix_max/pix_step) 
for i in range(pix_step-1):
    # print(i,pix_per_step*i+1,pix_per_step*(i+1))
    pix_grop = grouped[pix_per_step*i+1:pix_per_step*(i+1)]
    pix_hist = pix_grop.hist(t=t_edges)
    idstep_datas = sc.concat([step_data, pix_hist], 'id')
    # step_data = sc.merge(step_data, pix_hist)

# step_data
# pix_hist
# pix_grop
sc.show(pix_grop)

read in geometry informations

In [ ]:
f = h5py.File(filename)
xml  = str(f['entry1/instrument/instrument_xml/data'][...][0]).split('\\n')
xml


gets sample, source and detector positions. <br>
gets fast and slow axis of detectors

In [ ]:
comp = False
det = False
source = False
sample = False
sample_pos = [0,0,0]
source_pos = [0,0,0]
d_list = []
rot_l1 = []
fast_l = []
slow_l = []


for i in range(len(xml)):
    ls = xml[i].replace('<t',' ').replace('>',' ').replace('"',' ').replace('<',' ').replace('\\t',' ').split()
    # print(xml[i])
    
    if len(ls) >= 1:
        if ls[0] == 'component':
            det = False
            source = False
            sample = False
            comp = True
            if ls[2].split('-')[0] == 'MonNDtype':

                det = True
                d_list.append([int(ls[2].split('-')[1])])
            elif ls[2] == 'sourceMantid-type':
                source = True
            elif ls[2] == 'sampleMantid-type':
                sample = True
            comp = True
     #   if ls[1].split('-')[0] == 'type="MonNDtype':
            # print("1",ls)
    if len(ls) >= 1:
        if ls[0] == 'type':
            comp = False
    if len(ls) >= 1:
        if ls[0] == 'location' or ls[0] == 'location':
            # print("3",ls)
            if comp == True and det == True:
                # print("2",ls)
                xyz = [float(ls[2]),float(ls[4]),float(ls[6])]
                print("xyz", xyz)
                rot =float(ls[8]) 
                rot_xyz =[ float(ls[8]), float(ls[10]),float(ls[12]),float(ls[14])] 
                print("rotation of detector",rot, rot_xyz) 
                d_list[len(d_list)-1].append(xyz)
                rot_l1.append(rot_xyz)
            elif comp == True and source == True: 
              source_pos = [float(ls[2]),float(ls[4]),float(ls[6])]
            elif comp == True and sample == True: 
               sample_pos = np.array([float(ls[2]),float(ls[4]),float(ls[6])])
# print(len(d_list))
# print("sample and source position",sample_pos,source_pos)
# print("distance between sample and source",CDist2(source_pos, sample_pos))
# print("detector positions, relative to source at 0,0,0:",d_list)
# print("rotation list",rot_l)
#shift from rleative position to source to relative to sample
ds_l = []
sample_pos = sample_pos * [-1,-1,-1]
print("sample_pos",sample_pos)
for i in range(len(d_list)):
    det_pos = np.array([d_list[i][1][0],d_list[i][1][1],d_list[i][1][2]]) * [-1.0, -1.0,-1.0]
    

    rel_xyz= np.round(twoP_to_vec(sample_pos,det_pos),2)
    print("rel_xyz",rel_xyz)
    rel_xyz = rel_xyz # * [-1,-1,-1]
    print("sample to detector dist",len3dvec(rel_xyz))
    # rel_pos= twoP_to_vec(det_pos, sample_pos)
    # print("detector position",sample_pos,det_pos,rel_pos)
    
    # rel_xyz = [ d_list[i][1][0]-sample_pos[0], d_list[i][1][1]-sample_pos[1], (d_list[i][1][2]-sample_pos[2])]
    
    # print("relative position",rel_xyz)
    # rel_xyz = [ sample_pos[0]-d_list[i][1][0],sample_pos[1]-d_list[i][1][1], sample_pos[2]-d_list[i][1][2]]
    # print("position",rel_xyz,rel_pos)
    # print("relative position",rel_xyz)
    ds_l.append(rel_xyz)
vec_f = [-1,0,-0]
vec_s = [0,-1,0]
rot_l = [rot_l1[0],rot_l1[2],rot_l1[1]] #not corect but a temporary work around
for i in range(len(rot_l)):
    # if rot_l[i][2] < 0:
        # theta = np.radians(rot_l[i][0]) 
    # else:
    theta = np.radians(-rot_l[i][0])
    v = [rot_l[i][1],rot_l[i][2],rot_l[i][3]]
    v1 = [0,0,1]
    print("v",i,v,"theta", theta,rot_l[i][0])
    fast_vec = np.round(np.dot(rotation_matrix(v,  theta), vec_f),2)
    fast_v_round = np.array([np.round(fast_vec[0],1), np.round(fast_vec[1],1), np.round(fast_vec[2],1)])

    fast_l.append(fast_vec)
    slow_l.append(np.dot(rotation_matrix(v, theta), vec_s))
    # slow_l.append([0,1,0])

print("relative to sample position",ds_l)
print("fast axis",fast_l)
print("slow axis", slow_l)
print(rot_l)

manual seeting of the detector gementrie 

In [ ]:
#read in data
# ds_l = [[ 0.25,  0.25, -0.29], [ 0.29,  0.25, -0.25], [-0.29,  0.25,  0.25]]
# fast_l = [[-1.  ,  0.  , -0.01], [-0.01,  0.  ,  1.  ], [ 0.01,  0.  , -1.  ]]
# slow_l = [[ 0., -1.,  0.], [ 0., -1.,  0.],[ 0., -1.,  0.]]


ds_l = [[ -0.25,  -0.25, -0.29], [ -0.29,  -0.25, -0.25], [0.29,  -0.25,  0.25]]
fast_l = [[1.  ,  0.  , -0.01], [0.01,  0.  ,  1.  ], [ -0.01,  0.  , -1.  ]]
slow_l = [[ 0., 1.,  0.], [ 0., 1.,  0.],[ 0., 1.,  0.]]


gets crystal rotation

In [ ]:
print((f['entry1']['simulation']['Param'].keys()))
print((f['entry1']['simulation']['Param']['XtalPhiX']))
phix=float(list(str(f['entry1']['simulation']['Param']['XtalPhiX'][...][0]))[2])
phiy=float(list(str(f['entry1']['simulation']['Param']['XtalPhiY'][...][0]))[2])
phiz=float(list(str(f['entry1']['simulation']['Param']['XtalPhiZ'][...][0]))[2])
#str(phix[0])
#int(list(str(phix[0]))[2])
print(phix,phiy,phiz)
cor=[phix,phiy, phiz]
cryst_or = np.array(cor)
cryst_or

generates output file name out of input filename and number of  time bins

In [ ]:
no = filename.split('/')
print(no)
name_out= no[-1].split('.')[0]
print(name_out)
file_out = '/Users/justinbergmann/work_flow/test_out/'+name_out+'_'+str(t_step)+'_out.h5'
print(file_out)

In [ ]:
with h5py.File(file_out, 'w') as fo:
## create output nexus file
   fo.attrs['default'] = 'NMX_data'
   nxentry = fo.create_group('NMX_data')
   nxentry.attrs["NX_class"] = 'NXentry'
   nxentry.attrs['default'] = 'data'
   nxentry.attrs['name'] = 'NMX1'
   #nxentry.__setitem__('beamline','NMX')
   nxentry.__setitem__('name','NMX')
   nxentry.__setitem__('definition','TOFRAW')
   nxentry.attrs['name'] = "NMX"
   #nxentry.attrs['name'].__setattr__('name','NMX') 

#SAMPLE
   nx_sample = nxentry.create_group('NXsample')
   nx_sample.__setitem__('name','Single_crystal')


#Instrument
   nx_instrument = nxentry.create_group('NXinstrument')

   nx_detector = nxentry.create_group('NXdetector')
   det_origen = nx_detector.create_dataset('origen', data=ds_l) 
   det_origen.attrs['units'] = 'm'

   fast_axis = nx_detector.create_dataset('fast_axis', data=fast_l) 
   slow_axis = nx_detector.create_dataset('slow_axis', data=slow_l) 

   nx_beam = nxentry.create_group('NXbeam')


   
   proton = nxentry.create_dataset('proton_charge', data=weights.shape[0]/10000)    
   
   
   nx_det1 = nxentry.create_group('detector_1')   
   counts = nx_det1.create_dataset('counts', data=[group_t.values], compression="gzip", compression_opts=4)

   t_spectrum = nx_det1.create_dataset('t_bin', data=group_t.coords['t'].values, compression="gzip", compression_opts=4)
   t_spectrum.attrs['units'] = 'ms'
   t_spectrum.attrs['long_name'] = 't_bin TOF (ms)'

   pixel_id = nx_det1.create_dataset('pix_id', data=group_t.coords['id'].values, compression="gzip", compression_opts=4)
   pixel_id.attrs['units'] = ''
   pixel_id.attrs['long_name'] = 'pixel ID'


#SOURCE
   nx_inst = nxentry.create_group('instrument')
   nx_inst.attrs['nr_detector'] = len(fast_l)
   nx_source = nxentry.create_group('NXsource')
   nx_source.__setitem__('name','European Spallation Source')
   nx_source.__setitem__('short_name','ESS')
   nx_source.__setitem__('type','Spallation Neutron Source')
   nx_source.__setitem__('distance',-(CDist2(source_pos, sample_pos)))
   nx_source.__setitem__('probe','neutron')
   nx_source.__setitem__('target_material','W')






#    c_or = nxinst.create_dataset('crystal_orientation',data=cryst_or)
#    c_or.attrs['unit'] = 'degrees'
#    c_or.attrs['long_name'] = 'crystal orientation in Phi (degrees)'
    

   fo.close()
   del fo



delete some loaded lists


writes out some statistic of the reduction

In [ ]:
current, peak = tracemalloc.get_traced_memory()
print('2 current memory [MB]: {}, peak memory [MB]: {} '.format(round((current/(1024*1024)), 2), round((peak /(1024*1024) ), 2) ))
# stopping the library
# tracemalloc.stop()
print("neded time (h:mm:ss): ", datetime.now() - startTime)